# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

Dor Kremer </br>
dorkremer1597@gmail.com</br>

Michael Denis</br>
themoshe12@gmail.com


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# %pip install wn
# %python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# %pip install hebrew_tokenizer

In [5]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\Herr Yulie\Desktop\HIT\Text Analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8').replace({'f':0,'m':1})
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8').replace({'f':0,'m':1})

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",1
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,1
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,0
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",1
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,0
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,0
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,1
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,0


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

<b>In order to tokenize the stories we first have to get rid of the redundant words, words that are shorter than 2 characters and also words that contain characters that aren't Hebrew</b>

In [10]:
def removeRedundantTokens(tokens):
    res=[]
    for token in tokens:
        valid=True
        if len(token)<2:
            continue
        for ch in token:
            if (ord(ch) < ord("א") or ord(ch) > ord("ת")):
                valid=False
                break
        if valid:
            res.append(token)
    return res

<b>Now we can start processing the stories and split them into individual words</b>

In [11]:
def processingData(df, column):
    res=[]
    for text in df[column]:
        tokens=[]
        tokensPerText=ht.tokenize(text)
        for t in tokensPerText:
            tokens.append(t[1])
        cleanTokens=removeRedundantTokens(tokens)
        res.append(cleanTokens)
    newDf=df.copy()
    newDf["tokens"]=res
    return newDf

In [12]:
processed_df=processingData(df_train,"story")
processed_test=processingData(df_test,"story")

In [13]:
scores=[]
estimators=[]
params=[]
clfs=[]
vecs=[]
cvs=[]

<b>Now we are finally done with processing our data and we can move on to the machine learning part.</br>
We first define a Pipeline object with a vectorizer and a classifier. Then we pass that same Pipeline object into a GridSearch function with a list of different hyperparameters and from that we can determine the best hyperparameters for the vectorizer and classifier pair.</b>

<b>Count Vectorizer + Logistic Regression (cv=5)

In [14]:
cv_logReg_pipe = Pipeline([('countVectorizer', CountVectorizer()), ('log_reg_clf', linear_model.LogisticRegression())])
grid_params={'countVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
cv_logReg_search=GridSearchCV(cv_logReg_pipe, grid_params, cv=5, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
cv_logReg_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", cv_logReg_search.best_score_)
print("\nBest Estimator: ", cv_logReg_search.best_estimator_)
print("\nBest Parameters: ", cv_logReg_search.best_params_)
clfs.append("Logistic Regression")
vecs.append("Count Vectorizer")
cvs.append(5)
scores.append(cv_logReg_search.best_score_)
estimators.append(cv_logReg_search.best_estimator_)
params.append(cv_logReg_search.best_params_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('countVectorizer', CountVectorizer()),
                                       ('log_reg_clf', LogisticRegression())]),
             param_grid={'countVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.6620282085176806

Best Estimator:  Pipeline(steps=[('countVectorizer', CountVectorizer()),
                ('log_reg_clf', LogisticRegression())])

Best Parameters:  {'countVectorizer__ngram_range': (1, 1)}


<b>Count Vectorizer + Logistic Regression (cv=12)

In [15]:
cv_logReg_pipe = Pipeline([('countVectorizer', CountVectorizer()), ('log_reg_clf', linear_model.LogisticRegression())])
grid_params={'countVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
cv_logReg_search=GridSearchCV(cv_logReg_pipe, grid_params, cv=12, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
cv_logReg_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", cv_logReg_search.best_score_)
print("\nBest Estimator: ", cv_logReg_search.best_estimator_)
print("\nBest Parameters: ", cv_logReg_search.best_params_)
clfs.append("Logistic Regression")
vecs.append("Count Vectorizer")
cvs.append(12)
scores.append(cv_logReg_search.best_score_)
estimators.append(cv_logReg_search.best_estimator_)
params.append(cv_logReg_search.best_params_)

GridSearchCV(cv=12,
             estimator=Pipeline(steps=[('countVectorizer', CountVectorizer()),
                                       ('log_reg_clf', LogisticRegression())]),
             param_grid={'countVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.6466996634386426

Best Estimator:  Pipeline(steps=[('countVectorizer', CountVectorizer()),
                ('log_reg_clf', LogisticRegression())])

Best Parameters:  {'countVectorizer__ngram_range': (1, 1)}


<b>Tfidf Vectorizer + Logistic Regression (cv=5)

In [16]:
tfidf_logReg_pipe = Pipeline([('tfidfVectorizer', TfidfVectorizer()), ('log_reg_clf', linear_model.LogisticRegression())])
grid_params={'tfidfVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
tfidf_logReg_search=GridSearchCV(tfidf_logReg_pipe, grid_params, cv=5, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
tfidf_logReg_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", tfidf_logReg_search.best_score_)
print("\nBest Estimator: ", tfidf_logReg_search.best_estimator_)
print("\nBest Parameters: ", tfidf_logReg_search.best_params_)
clfs.append("Logistic Regression")
vecs.append("Tfidf Vectorizer")
cvs.append(5)
scores.append(tfidf_logReg_search.best_score_)
estimators.append(tfidf_logReg_search.best_estimator_)
params.append(tfidf_logReg_search.best_params_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                                       ('log_reg_clf', LogisticRegression())]),
             param_grid={'tfidfVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.43298340190097884

Best Estimator:  Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                ('log_reg_clf', LogisticRegression())])

Best Parameters:  {'tfidfVectorizer__ngram_range': (1, 1)}


<b>Tfidf Vectorizer + Logistic Regression (cv=12)

In [17]:
tfidf_logReg_pipe = Pipeline([('tfidfVectorizer', TfidfVectorizer()), ('log_reg_clf', linear_model.LogisticRegression())])
grid_params={'tfidfVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
tfidf_logReg_search=GridSearchCV(tfidf_logReg_pipe, grid_params, cv=12, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
tfidf_logReg_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", tfidf_logReg_search.best_score_)
print("\nBest Estimator: ", tfidf_logReg_search.best_estimator_)
print("\nBest Parameters: ", tfidf_logReg_search.best_params_)
clfs.append("Logistic Regression")
vecs.append("Tfidf Vectorizer")
cvs.append(12)
scores.append(tfidf_logReg_search.best_score_)
estimators.append(tfidf_logReg_search.best_estimator_)
params.append(tfidf_logReg_search.best_params_)

GridSearchCV(cv=12,
             estimator=Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                                       ('log_reg_clf', LogisticRegression())]),
             param_grid={'tfidfVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.4329843187641353

Best Estimator:  Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                ('log_reg_clf', LogisticRegression())])

Best Parameters:  {'tfidfVectorizer__ngram_range': (1, 1)}


<b>Count Vectorizer + KNN (cv=5)

In [18]:
cv_knn_pipe = Pipeline([('countVectorizer', CountVectorizer()), ('knn_clf', KNeighborsClassifier())])
grid_params={'knn_clf__n_neighbors':[3,5,7], 
             'countVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
cv_knn_search=GridSearchCV(cv_knn_pipe,grid_params, cv=5, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
cv_knn_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", cv_knn_search.best_score_)
print("\nBest Estimator: ", cv_knn_search.best_estimator_)
print("\nBest Parameters: ", cv_knn_search.best_params_)
clfs.append("KNN")
vecs.append("Count Vectorizer")
cvs.append(5)
scores.append(cv_knn_search.best_score_)
estimators.append(cv_knn_search.best_estimator_)
params.append(cv_knn_search.best_params_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('countVectorizer', CountVectorizer()),
                                       ('knn_clf', KNeighborsClassifier())]),
             param_grid={'countVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)],
                         'knn_clf__n_neighbors': [3, 5, 7]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.5383986409181778

Best Estimator:  Pipeline(steps=[('countVectorizer', CountVectorizer()),
                ('knn_clf', KNeighborsClassifier(n_neighbors=3))])

Best Parameters:  {'countVectorizer__ngram_range': (1, 1), 'knn_clf__n_neighbors': 3}


<b>Count Vectorizer + KNN (cv=12)

In [19]:
cv_knn_pipe = Pipeline([('countVectorizer', CountVectorizer()), ('knn_clf', KNeighborsClassifier())])
grid_params={'knn_clf__n_neighbors':[3,5,7], 
             'countVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
cv_knn_search=GridSearchCV(cv_knn_pipe,grid_params, cv=12, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
cv_knn_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", cv_knn_search.best_score_)
print("\nBest Estimator: ", cv_knn_search.best_estimator_)
print("\nBest Parameters: ", cv_knn_search.best_params_)
clfs.append("KNN")
vecs.append("Count Vectorizer")
cvs.append(12)
scores.append(cv_knn_search.best_score_)
estimators.append(cv_knn_search.best_estimator_)
params.append(cv_knn_search.best_params_)

GridSearchCV(cv=12,
             estimator=Pipeline(steps=[('countVectorizer', CountVectorizer()),
                                       ('knn_clf', KNeighborsClassifier())]),
             param_grid={'countVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)],
                         'knn_clf__n_neighbors': [3, 5, 7]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.5241347774134925

Best Estimator:  Pipeline(steps=[('countVectorizer', CountVectorizer()),
                ('knn_clf', KNeighborsClassifier(n_neighbors=3))])

Best Parameters:  {'countVectorizer__ngram_range': (1, 1), 'knn_clf__n_neighbors': 3}


<b>Tfidf Vectorizer + KNN (cv=5)

In [20]:
tfidf_knn_pipe = Pipeline([('tfidfVectorizer', TfidfVectorizer()), ('knn_clf', KNeighborsClassifier())])
grid_params={'knn_clf__n_neighbors':[3,5,7], 'tfidfVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
tfidf_knn_search=GridSearchCV(tfidf_knn_pipe,grid_params, cv=5, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
tfidf_knn_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", tfidf_knn_search.best_score_)
print("\nBest Estimator: ", tfidf_knn_search.best_estimator_)
print("\nBest Parameters: ", tfidf_knn_search.best_params_)
clfs.append("KNN")
vecs.append("Tfidf Vectorizer")
cvs.append(5)
scores.append(tfidf_knn_search.best_score_)
estimators.append(tfidf_knn_search.best_estimator_)
params.append(tfidf_knn_search.best_params_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                                       ('knn_clf', KNeighborsClassifier())]),
             param_grid={'knn_clf__n_neighbors': [3, 5, 7],
                         'tfidfVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.6243469187901602

Best Estimator:  Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer(ngram_range=(1, 3))),
                ('knn_clf', KNeighborsClassifier())])

Best Parameters:  {'knn_clf__n_neighbors': 5, 'tfidfVectorizer__ngram_range': (1, 3)}


<b>Tfidf Vectorizer + KNN (cv=12)

In [21]:
tfidf_knn_pipe = Pipeline([('tfidfVectorizer', TfidfVectorizer()), ('knn_clf', KNeighborsClassifier())])
grid_params={'knn_clf__n_neighbors':[3,5,7], 'tfidfVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
tfidf_knn_search=GridSearchCV(tfidf_knn_pipe,grid_params, cv=12, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
tfidf_knn_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", tfidf_knn_search.best_score_)
print("\nBest Estimator: ", tfidf_knn_search.best_estimator_)
print("\nBest Parameters: ", tfidf_knn_search.best_params_)
clfs.append("KNN")
vecs.append("Tfidf Vectorizer")
cvs.append(12)
scores.append(tfidf_knn_search.best_score_)
estimators.append(tfidf_knn_search.best_estimator_)
params.append(tfidf_knn_search.best_params_)

GridSearchCV(cv=12,
             estimator=Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                                       ('knn_clf', KNeighborsClassifier())]),
             param_grid={'knn_clf__n_neighbors': [3, 5, 7],
                         'tfidfVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.6409322900973061

Best Estimator:  Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                ('knn_clf', KNeighborsClassifier())])

Best Parameters:  {'knn_clf__n_neighbors': 5, 'tfidfVectorizer__ngram_range': (1, 2)}


<b>Count Vectorizer + Decision Tree (cv=5)

In [22]:
cv_dt_pipe = Pipeline([('countVectorizer', CountVectorizer()), ('dt_clf', DecisionTreeClassifier())])
grid_params={'dt_clf__max_depth': [3, 5, 7], 'dt_clf__criterion': ['gini', 'entropy'], 'countVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
cv_dt_search=GridSearchCV(cv_dt_pipe,grid_params, cv=5, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
cv_dt_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", cv_dt_search.best_score_)
print("\nBest Estimator: ", cv_dt_search.best_estimator_)
print("\nBest Parameters: ", cv_dt_search.best_params_)
clfs.append("Decision Tree")
vecs.append("Count Vectorizer")
cvs.append(5)
scores.append(cv_dt_search.best_score_)
estimators.append(cv_dt_search.best_estimator_)
params.append(cv_dt_search.best_params_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('countVectorizer', CountVectorizer()),
                                       ('dt_clf', DecisionTreeClassifier())]),
             param_grid={'countVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)],
                         'dt_clf__criterion': ['gini', 'entropy'],
                         'dt_clf__max_depth': [3, 5, 7]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.5503319161890481

Best Estimator:  Pipeline(steps=[('countVectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('dt_clf', DecisionTreeClassifier(max_depth=7))])

Best Parameters:  {'countVectorizer__ngram_range': (1, 2), 'dt_clf__criterion': 'gini', 'dt_clf__max_depth': 7}


<b>Count Vectorizer + Decision Tree (cv=12)

In [23]:
cv_dt_pipe = Pipeline([('countVectorizer', CountVectorizer()), ('dt_clf', DecisionTreeClassifier())])
grid_params={'dt_clf__max_depth': [3, 5, 7], 'dt_clf__criterion': ['gini', 'entropy'], 'countVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
cv_dt_search=GridSearchCV(cv_dt_pipe,grid_params, cv=12, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
cv_dt_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", cv_dt_search.best_score_)
print("\nBest Estimator: ", cv_dt_search.best_estimator_)
print("\nBest Parameters: ", cv_dt_search.best_params_)
clfs.append("Decision Tree")
vecs.append("Count Vectorizer")
cvs.append(12)
scores.append(cv_dt_search.best_score_)
estimators.append(cv_dt_search.best_estimator_)
params.append(cv_dt_search.best_params_)

GridSearchCV(cv=12,
             estimator=Pipeline(steps=[('countVectorizer', CountVectorizer()),
                                       ('dt_clf', DecisionTreeClassifier())]),
             param_grid={'countVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)],
                         'dt_clf__criterion': ['gini', 'entropy'],
                         'dt_clf__max_depth': [3, 5, 7]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.5750501412439768

Best Estimator:  Pipeline(steps=[('countVectorizer', CountVectorizer()),
                ('dt_clf', DecisionTreeClassifier(max_depth=7))])

Best Parameters:  {'countVectorizer__ngram_range': (1, 1), 'dt_clf__criterion': 'gini', 'dt_clf__max_depth': 7}


<b>Tfidf Vectorizer + Decision Tree (cv=5)

In [24]:
tfidf_dt_pipe = Pipeline([('tfidfVectorizer', TfidfVectorizer()), ('dt_clf', DecisionTreeClassifier())])
grid_params={'dt_clf__max_depth': [3, 5, 7], 'dt_clf__criterion': ['gini', 'entropy'], 'tfidfVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
tfidf_dt_search=GridSearchCV(tfidf_dt_pipe,grid_params, cv=5, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
tfidf_dt_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", tfidf_dt_search.best_score_)
print("\nBest Estimator: ", tfidf_dt_search.best_estimator_)
print("\nBest Parameters: ", tfidf_dt_search.best_params_)
clfs.append("Decision Tree")
vecs.append("Tfidf Vectorizer")
cvs.append(5)
scores.append(tfidf_dt_search.best_score_)
estimators.append(tfidf_dt_search.best_estimator_)
params.append(tfidf_dt_search.best_params_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                                       ('dt_clf', DecisionTreeClassifier())]),
             param_grid={'dt_clf__criterion': ['gini', 'entropy'],
                         'dt_clf__max_depth': [3, 5, 7],
                         'tfidfVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.5870010824231441

Best Estimator:  Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                ('dt_clf', DecisionTreeClassifier(max_depth=7))])

Best Parameters:  {'dt_clf__criterion': 'gini', 'dt_clf__max_depth': 7, 'tfidfVectorizer__ngram_range': (1, 2)}


<b>Tfidf Vectorizer + Decision Tree (cv=12)

In [25]:
tfidf_dt_pipe = Pipeline([('tfidfVectorizer', TfidfVectorizer()), ('dt_clf', DecisionTreeClassifier())])
grid_params={'dt_clf__max_depth': [3, 5, 7], 'dt_clf__criterion': ['gini', 'entropy'], 'tfidfVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
tfidf_dt_search=GridSearchCV(tfidf_dt_pipe,grid_params, cv=12, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
tfidf_dt_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", tfidf_dt_search.best_score_)
print("\nBest Estimator: ", tfidf_dt_search.best_estimator_)
print("\nBest Parameters: ", tfidf_dt_search.best_params_)
clfs.append("Decision Tree")
vecs.append("Tfidf Vectorizer")
cvs.append(12)
scores.append(tfidf_dt_search.best_score_)
estimators.append(tfidf_dt_search.best_estimator_)
params.append(tfidf_dt_search.best_params_)

GridSearchCV(cv=12,
             estimator=Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                                       ('dt_clf', DecisionTreeClassifier())]),
             param_grid={'dt_clf__criterion': ['gini', 'entropy'],
                         'dt_clf__max_depth': [3, 5, 7],
                         'tfidfVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.6008791536474979

Best Estimator:  Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer(ngram_range=(1, 3))),
                ('dt_clf',
                 DecisionTreeClassifier(criterion='entropy', max_depth=7))])

Best Parameters:  {'dt_clf__criterion': 'entropy', 'dt_clf__max_depth': 7, 'tfidfVectorizer__ngram_range': (1, 3)}


<b>Count Vectorizer + Multinomial Naive Bayes (cv=5)

In [26]:
cv_mnb_pipe = Pipeline([('countVectorizer', CountVectorizer()), ('mnb_clf', MultinomialNB())])
grid_params={'mnb_clf__alpha': [0.1, 0.5, 1, 10], 'countVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
cv_mnb_search=GridSearchCV(cv_mnb_pipe,grid_params, cv=5, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
cv_mnb_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", cv_mnb_search.best_score_)
print("\nBest Estimator: ", cv_mnb_search.best_estimator_)
print("\nBest Parameters: ", cv_mnb_search.best_params_)
clfs.append("Multinomial Naive Bayes")
vecs.append("Count Vectorizer")
cvs.append(5)
scores.append(cv_mnb_search.best_score_)
estimators.append(cv_mnb_search.best_estimator_)
params.append(cv_mnb_search.best_params_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('countVectorizer', CountVectorizer()),
                                       ('mnb_clf', MultinomialNB())]),
             param_grid={'countVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)],
                         'mnb_clf__alpha': [0.1, 0.5, 1, 10]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.5537407997214999

Best Estimator:  Pipeline(steps=[('countVectorizer', CountVectorizer()),
                ('mnb_clf', MultinomialNB(alpha=0.1))])

Best Parameters:  {'countVectorizer__ngram_range': (1, 1), 'mnb_clf__alpha': 0.1}


<b>Count Vectorizer + Multinomial Naive Bayes (cv=12)

In [27]:
cv_mnb_pipe = Pipeline([('countVectorizer', CountVectorizer()), ('mnb_clf', MultinomialNB())])
grid_params={'mnb_clf__alpha': [0.1, 0.5, 1, 10], 'countVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
cv_mnb_search=GridSearchCV(cv_mnb_pipe,grid_params, cv=12, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
cv_mnb_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", cv_mnb_search.best_score_)
print("\nBest Estimator: ", cv_mnb_search.best_estimator_)
print("\nBest Parameters: ", cv_mnb_search.best_params_)
clfs.append("Multinomial Naive Bayes")
vecs.append("Count Vectorizer")
cvs.append(12)
scores.append(cv_mnb_search.best_score_)
estimators.append(cv_mnb_search.best_estimator_)
params.append(cv_mnb_search.best_params_)

GridSearchCV(cv=12,
             estimator=Pipeline(steps=[('countVectorizer', CountVectorizer()),
                                       ('mnb_clf', MultinomialNB())]),
             param_grid={'countVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)],
                         'mnb_clf__alpha': [0.1, 0.5, 1, 10]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.5612819825855629

Best Estimator:  Pipeline(steps=[('countVectorizer', CountVectorizer()),
                ('mnb_clf', MultinomialNB(alpha=0.1))])

Best Parameters:  {'countVectorizer__ngram_range': (1, 1), 'mnb_clf__alpha': 0.1}


<b>Tfidf Vectorizer + Multinomial Naive Bayes (cv=5)

In [28]:
tfidf_mnb_pipe = Pipeline([('tfidfVectorizer', TfidfVectorizer()), ('mnb_clf', MultinomialNB())])
grid_params={'mnb_clf__alpha': [0.1, 0.5, 1, 10], 'tfidfVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
tfidf_mnb_search=GridSearchCV(tfidf_mnb_pipe,grid_params, cv=5, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
tfidf_mnb_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", tfidf_mnb_search.best_score_)
print("\nBest Estimator: ", tfidf_mnb_search.best_estimator_)
print("\nBest Parameters: ", tfidf_mnb_search.best_params_)
clfs.append("Multinomial Naive Bayes")
vecs.append("Tfidf Vectorizer")
cvs.append(5)
scores.append(tfidf_mnb_search.best_score_)
estimators.append(tfidf_mnb_search.best_estimator_)
params.append(tfidf_mnb_search.best_params_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                                       ('mnb_clf', MultinomialNB())]),
             param_grid={'mnb_clf__alpha': [0.1, 0.5, 1, 10],
                         'tfidfVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.46581963280872446

Best Estimator:  Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                ('mnb_clf', MultinomialNB(alpha=0.1))])

Best Parameters:  {'mnb_clf__alpha': 0.1, 'tfidfVectorizer__ngram_range': (1, 2)}


<b>Tfidf Vectorizer + Multinomial Naive Bayes (cv=12)

In [29]:
tfidf_mnb_pipe = Pipeline([('tfidfVectorizer', TfidfVectorizer()), ('mnb_clf', MultinomialNB())])
grid_params={'mnb_clf__alpha': [0.1, 0.5, 1, 10], 'tfidfVectorizer__ngram_range':[(1,1),(1,2),(1,3)]}
tfidf_mnb_search=GridSearchCV(tfidf_mnb_pipe,grid_params, cv=12, scoring=metrics.make_scorer(metrics.f1_score, average='macro'))
tfidf_mnb_search.fit(processed_df["story"], processed_df["gender"])
print("Best F1 Score:", tfidf_mnb_search.best_score_)
print("\nBest Estimator: ", tfidf_mnb_search.best_estimator_)
print("\nBest Parameters: ", tfidf_mnb_search.best_params_)
clfs.append("Multinomial Naive Bayes")
vecs.append("Tfidf Vectorizer")
cvs.append(12)
scores.append(tfidf_mnb_search.best_score_)
estimators.append(tfidf_mnb_search.best_estimator_)
params.append(tfidf_mnb_search.best_params_)

GridSearchCV(cv=12,
             estimator=Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer()),
                                       ('mnb_clf', MultinomialNB())]),
             param_grid={'mnb_clf__alpha': [0.1, 0.5, 1, 10],
                         'tfidfVectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)]},
             scoring=make_scorer(f1_score, average=macro))

Best F1 Score: 0.47471723617556955

Best Estimator:  Pipeline(steps=[('tfidfVectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                ('mnb_clf', MultinomialNB(alpha=0.1))])

Best Parameters:  {'mnb_clf__alpha': 0.1, 'tfidfVectorizer__ngram_range': (1, 2)}


<b>Now we can create a dataframe from the results we got from the different experiments to see which one is the most accurate.

In [30]:
results=pd.DataFrame({"classifier":clfs,"vectorizer":vecs,"cv_split":cvs,"score":scores,"estimator":estimators,"parameters":params})
results.sort_values(by=["score"], ascending=False )

,classifier,vectorizer,cv_split,score,estimator,parameters
0,Logistic Regression,Count Vectorizer,5,0.662,"(CountVectorizer(), LogisticRegression())","{'countVectorizer__ngram_range': (1, 1)}"
1,Logistic Regression,Count Vectorizer,12,0.647,"(CountVectorizer(), LogisticRegression())","{'countVectorizer__ngram_range': (1, 1)}"
7,KNN,Tfidf Vectorizer,12,0.641,"(TfidfVectorizer(ngram_range=(1, 2)), KNeighbo...","{'knn_clf__n_neighbors': 5, 'tfidfVectorizer__..."
6,KNN,Tfidf Vectorizer,5,0.624,"(TfidfVectorizer(ngram_range=(1, 3)), KNeighbo...","{'knn_clf__n_neighbors': 5, 'tfidfVectorizer__..."
11,Decision Tree,Tfidf Vectorizer,12,0.601,"(TfidfVectorizer(ngram_range=(1, 3)), Decision...","{'dt_clf__criterion': 'entropy', 'dt_clf__max_..."
10,Decision Tree,Tfidf Vectorizer,5,0.587,"(TfidfVectorizer(ngram_range=(1, 2)), Decision...","{'dt_clf__criterion': 'gini', 'dt_clf__max_dep..."
9,Decision Tree,Count Vectorizer,12,0.575,"(CountVectorizer(), DecisionTreeClassifier(max...","{'countVectorizer__ngram_range': (1, 1), 'dt_c..."
13,Multinomial Naive Bayes,Count Vectorizer,12,0.561,"(CountVectorizer(), MultinomialNB(alpha=0.1))","{'countVectorizer__ngram_range': (1, 1), 'mnb_..."
12,Multinomial Naive Bayes,Count Vectorizer,5,0.554,"(CountVectorizer(), MultinomialNB(alpha=0.1))","{'countVectorizer__ngram_range': (1, 1), 'mnb_..."
8,Decision Tree,Count Vectorizer,5,0.550,"(CountVectorizer(ngram_range=(1, 2)), Decision...","{'countVectorizer__ngram_range': (1, 2), 'dt_c..."


<b>As we can easily see, the <u>Count Vectorizer and Logistic Regression</u> pair had the highest F1 score.
Now we use the pipeline with said pair to try and predict the genders of the test stories.

In [31]:
cv_logReg_pipe.fit(processed_df["story"], processed_df["gender"])
prediction = cv_logReg_pipe.predict(processed_test['story'])
df_predicted=pd.DataFrame({'test_story_id': processed_test['test_example_id'], 'predicted_category': prediction})
df_predicted['predicted_category']=df_predicted['predicted_category'].replace({1:"m",0:"f"})
df_predicted["predicted_category"].value_counts()
pd.concat([df_predicted.head(5), df_predicted.tail(5)])

Pipeline(steps=[('countVectorizer', CountVectorizer()),
                ('log_reg_clf', LogisticRegression())])

m    282
f     41
Name: predicted_category, dtype: int64

,test_story_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [32]:
df_predicted.to_csv('classification_results.csv',index=False)